# Analysis Test 

## Question:
1. We would like to know whether users in different User ages (User Ages = Last Service Date - First Service Date) have different serviceid cluster?
2. We would like to know which serviceid we can cross-sales to users?
3. Any ideas that you want to analyze more with those data? 

# **About the Dataset**:
_Note: The provided test question is not clear about the data, therefore, this is my assumption for better analysis_.
- Servicedid: The identity of the transaction
- Date: the date that transaction is performed
- User_id: The identity of the individual customer
- User_age: the difference between Last Service Date and First Service Date.

# **Process**:
Using Python to prepare and process the data.

In [2]:
# Data processing
import numpy as np # data arrays
import pandas as pd # data structure and data analysis
import functools
import datetime as dt # date time


from mlxtend.frequent_patterns import apriori # Data pattern exploration
from mlxtend.frequent_patterns import association_rules # Association rules conversion

In [4]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_b7209d1e410e48e798bf539a04efcb4c = 'https://s3.eu.cloud-object-storage.appdomain.cloud'
else:
    endpoint_b7209d1e410e48e798bf539a04efcb4c = 'https://s3.private.eu.cloud-object-storage.appdomain.cloud'

client_b7209d1e410e48e798bf539a04efcb4c = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='bvxv4cFUZ5Hu2Em_-AQIejU32mMGEpuc1YNv4wlwynmL',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_b7209d1e410e48e798bf539a04efcb4c)

body = client_b7209d1e410e48e798bf539a04efcb4c.get_object(Bucket='pythonbasicsfordataproject-donotdelete-pr-2lygblpvjr9idu',Key='test_process.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df1 = pd.read_csv(body)
df1.head()


,User_id,FirstServiceid,SecondServiceid,FirstServiceDate,SecondServiceDate,LastServiceid,LastServiceDate,TotalService
0,2464231,18,667,7-Jan-18,16-Jan-18,982,23-Jul-18,10
1,3676235,273,481,5-Jan-18,6-Jan-18,355,12-Apr-18,9
2,4958104,946,310,22-Jan-18,22-Jan-18,268,30-Jun-18,7
3,11642209,20,666,1-Jan-18,2-Jan-18,666,29-Jun-18,14
4,18246539,984,269,22-Mar-18,22-Mar-18,666,1-Apr-18,4


In [5]:

body = client_b7209d1e410e48e798bf539a04efcb4c.get_object(Bucket='pythonbasicsfordataproject-donotdelete-pr-2lygblpvjr9idu',Key='Test_TTS Phan tich Du lieu.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df2 = pd.read_csv(body)
df2.head()


,Serviceid,Date,User_Id
0,2,23-May-17,38411702
1,2,12-Jun-17,38411702
2,2,24-Aug-17,38411702
3,2,25-Sep-17,38411702
4,667,01-Jan-18,20831230


In [6]:
df2.rename(columns = {'User_Id':'User_id'}, inplace=True)
df2.head()

,Serviceid,Date,User_id
0,2,23-May-17,38411702
1,2,12-Jun-17,38411702
2,2,24-Aug-17,38411702
3,2,25-Sep-17,38411702
4,667,01-Jan-18,20831230


## Analysis question 1:
We would like to know whether users in different MoMo ages have different serviceid cluster?

In [7]:
#Merge df1 and df2 into one datafram dfs for further analysis
dfs = [df1, df2]
UserTable = functools.reduce(lambda left,right: pd.merge(left,right,on='User_id', how='outer'), dfs)
UserTable.dropna(inplace=True)

In [8]:
UserTable.head() # print to confirm

,User_id,FirstServiceid,SecondServiceid,FirstServiceDate,SecondServiceDate,LastServiceid,LastServiceDate,TotalService,Serviceid,Date
0,2464231,18.0,667.0,7-Jan-18,16-Jan-18,982.0,23-Jul-18,10.0,18,07-Jan-18
1,2464231,18.0,667.0,7-Jan-18,16-Jan-18,982.0,23-Jul-18,10.0,667,16-Jan-18
2,2464231,18.0,667.0,7-Jan-18,16-Jan-18,982.0,23-Jul-18,10.0,333,16-Jan-18
3,2464231,18.0,667.0,7-Jan-18,16-Jan-18,982.0,23-Jul-18,10.0,667,26-Jan-18
4,2464231,18.0,667.0,7-Jan-18,16-Jan-18,982.0,23-Jul-18,10.0,333,26-Jan-18


In [9]:
#convert all service date to datatime64 dtype
UserTable['FirstServiceDate'] = pd.to_datetime(UserTable['FirstServiceDate'])
UserTable['SecondServiceDate'] = pd.to_datetime(UserTable['SecondServiceDate'])
UserTable['LastServiceDate'] = pd.to_datetime(UserTable['LastServiceDate'])
UserTable['Date'] = pd.to_datetime(UserTable['Date'])

# re-print information to confirm
UserTable.info()

# print 1st 5 rows to confirm
UserTable.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1463 entries, 0 to 1462
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   User_id            1463 non-null   int64         
 1   FirstServiceid     1463 non-null   float64       
 2   SecondServiceid    1463 non-null   float64       
 3   FirstServiceDate   1463 non-null   datetime64[ns]
 4   SecondServiceDate  1463 non-null   datetime64[ns]
 5   LastServiceid      1463 non-null   float64       
 6   LastServiceDate    1463 non-null   datetime64[ns]
 7   TotalService       1463 non-null   float64       
 8   Serviceid          1463 non-null   int64         
 9   Date               1463 non-null   datetime64[ns]
dtypes: datetime64[ns](4), float64(4), int64(2)
memory usage: 125.7 KB


,User_id,FirstServiceid,SecondServiceid,FirstServiceDate,SecondServiceDate,LastServiceid,LastServiceDate,TotalService,Serviceid,Date
0,2464231,18.0,667.0,2018-01-07,2018-01-16,982.0,2018-07-23,10.0,18,2018-01-07
1,2464231,18.0,667.0,2018-01-07,2018-01-16,982.0,2018-07-23,10.0,667,2018-01-16
2,2464231,18.0,667.0,2018-01-07,2018-01-16,982.0,2018-07-23,10.0,333,2018-01-16
3,2464231,18.0,667.0,2018-01-07,2018-01-16,982.0,2018-07-23,10.0,667,2018-01-26
4,2464231,18.0,667.0,2018-01-07,2018-01-16,982.0,2018-07-23,10.0,333,2018-01-26


In [10]:
#Calculate the User_age
UserTable['User_age'] = 1 + (UserTable['LastServiceDate']-UserTable['FirstServiceDate']).dt.days

In [11]:
# Split users into 5 categories of 30_age, 100_age, 200_age, 300_age, 300_plus_age

bins = [-1, 30, 100, 200, 300, np.inf]
names = ['30_age', '100_age', '200_age', '300_age', '300_plus_age']
UserTable['Age_bin'] = pd.cut(UserTable['User_age'], bins, labels = names)

#Print the first 5 rows to confirm
UserTable.head()

,User_id,FirstServiceid,SecondServiceid,FirstServiceDate,SecondServiceDate,LastServiceid,LastServiceDate,TotalService,Serviceid,Date,User_age,Age_bin
0,2464231,18.0,667.0,2018-01-07,2018-01-16,982.0,2018-07-23,10.0,18,2018-01-07,198,200_age
1,2464231,18.0,667.0,2018-01-07,2018-01-16,982.0,2018-07-23,10.0,667,2018-01-16,198,200_age
2,2464231,18.0,667.0,2018-01-07,2018-01-16,982.0,2018-07-23,10.0,333,2018-01-16,198,200_age
3,2464231,18.0,667.0,2018-01-07,2018-01-16,982.0,2018-07-23,10.0,667,2018-01-26,198,200_age
4,2464231,18.0,667.0,2018-01-07,2018-01-16,982.0,2018-07-23,10.0,333,2018-01-26,198,200_age


In [12]:
# Grouping customers bases on the group of Age_bin and servicedid, 
# in order to know which services were used most by each group

UserAgg = UserTable.groupby(['Age_bin','Serviceid'])[['User_id']].count().dropna()
UserAgg

User_id
Age_bin      Serviceid         
30_age       2                0
             4                0
             10               0
             11               0
             12               0
...                         ...
300_plus_age 3976             0
             3995             0
             3998             0
             4095             0
             4483             0

[395 rows x 1 columns]

In [13]:
# Since the result is not satisfied. We group by the first level of the index:
g = UserAgg['User_id'].groupby(level=0, group_keys=False)

# Sort 'order' each group and take the first five elements:
g.nlargest(5)

Age_bin       Serviceid
30_age        487           9
              1014          9
              299           4
              47            2
              65            2
100_age       333          24
              667          24
              981          13
              1014         11
              487           9
200_age       981          75
              20           70
              1014         58
              271          46
              326          42
300_age       18           93
              667          65
              333          64
              981          60
              268          49
300_plus_age  981           9
              19            8
              1014          5
              2             4
              271           4
Name: User_id, dtype: int64

### **Observation**
- Users in 30_age group tend to use Serviceid of 487, 1014, 299, 47, 65
- Users in 100_age group tend to use Serviceid of 333, 667, 981, 1014, 487
- Users in 200_age group tend to use Serviceid of 981, 20, 1014, 271, 326
- Users in 300_age group tend to use Serviceid of 18, 667, 333, 981, 268
- Users in 300_plus_age group tend to use Serviceid of 981, 19, 1014, 2, 271


## Analysis question 2:
We would like to know which serviceid we can cross-sales to users?

In [15]:
UserTable.head()

,User_id,FirstServiceid,SecondServiceid,FirstServiceDate,SecondServiceDate,LastServiceid,LastServiceDate,TotalService,Serviceid,Date,User_age,Age_bin
0,2464231,18.0,667.0,2018-01-07,2018-01-16,982.0,2018-07-23,10.0,18,2018-01-07,198,200_age
1,2464231,18.0,667.0,2018-01-07,2018-01-16,982.0,2018-07-23,10.0,667,2018-01-16,198,200_age
2,2464231,18.0,667.0,2018-01-07,2018-01-16,982.0,2018-07-23,10.0,333,2018-01-16,198,200_age
3,2464231,18.0,667.0,2018-01-07,2018-01-16,982.0,2018-07-23,10.0,667,2018-01-26,198,200_age
4,2464231,18.0,667.0,2018-01-07,2018-01-16,982.0,2018-07-23,10.0,333,2018-01-26,198,200_age


In [16]:
Cross_sales = UserTable[['User_id', 'Date', 'Serviceid']]
Cross_sales.head()

,User_id,Date,Serviceid
0,2464231,2018-01-07,18
1,2464231,2018-01-16,667
2,2464231,2018-01-16,333
3,2464231,2018-01-26,667
4,2464231,2018-01-26,333


_Note:_
I plan to apply the Association Rules and Apriori Algorithm in this section to find number of unique serviceids per day for each customer then we can cross-sales the items in combo.  
- Measure: Support, Confidence and Lift of each Userid&Date and the Serviceid. 
- Source: https://towardsdatascience.com/association-rules-2-aa9a77241654
First, we find the serviceid_set of each userid per day, then put the serviceid in combo for each user_age group based on the Maximal frequent serviceid_set.

**The results are presented like this:**
If a user converts Serviceid_333, we suggest them the Serviceid_667 with Support of 20%, Confidence of 100%, Lift being greater than 1.
If a user converts Serviceid_667, we suggest them the Serviceid_333 with Support of 20%, Confidence of 91.8%, Lift being greater than 1.
If a user converts Serviceid_981, we suggest them the Serviceid_1014 with Support of 16.9%, Confidence of 61%, Lift being greater than 1.


_Limitation:_
My Python skill is still not well to do the complex syntax, I just find errors and cannot fix it. I'll attach my syntax below. Since the due time is over, I have to submit the task. Nevertheless, I'll try this method in Excel as well.


In [32]:
Cross_sales['User_id & Date'] = pd.DataFrame(Cross_sales['User_id'].astype('str') + ' ' + Cross_sales['Date']. astype('str'))
Cross_sales


<ipython-input-32-04c54ce13855>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Cross_sales['User_id & Date'] = pd.DataFrame(Cross_sales['User_id'].astype('str') + ' ' + Cross_sales['Date']. astype('str'))


,User_id,Date,Serviceid,User_id & Date
0,2464231,2018-01-07,18,2464231 2018-01-07
1,2464231,2018-01-16,667,2464231 2018-01-16
2,2464231,2018-01-16,333,2464231 2018-01-16
3,2464231,2018-01-26,667,2464231 2018-01-26
4,2464231,2018-01-26,333,2464231 2018-01-26
...,...,...,...,...
1458,40750022,2018-01-29,47,40750022 2018-01-29
1459,40750022,2018-02-14,54,40750022 2018-02-14
1460,40750022,2018-02-14,13,40750022 2018-02-14
1461,40750022,2018-05-30,667,40750022 2018-05-30


In [36]:
Cross_sales.drop(['User_id', 'Date'], axis=1, inplace=True)

/opt/conda/envs/Python-3.8-main/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [37]:
Cross_sales.head(10)

,Serviceid,User_id & Date
0,18,2464231 2018-01-07
1,667,2464231 2018-01-16
2,333,2464231 2018-01-16
3,667,2464231 2018-01-26
4,333,2464231 2018-01-26
5,1366,2464231 2018-02-06
6,667,2464231 2018-03-26
7,667,2464231 2018-04-26
8,667,2464231 2018-05-25
9,268,2464231 2018-06-22


In [42]:
Cross_sales['Serviceid'] = pd.DataFrame(Cross_sales['Serviceid']).astype('str')

basket = pd.get_dummies(Cross_sales)
basket.head()


<ipython-input-42-aae2f54b04a9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Cross_sales['Serviceid'] = pd.DataFrame(Cross_sales['Serviceid']).astype('str')


,Serviceid_10,Serviceid_1014,Serviceid_1046,Serviceid_1086,Serviceid_11,Serviceid_1136,Serviceid_1166,Serviceid_1186,Serviceid_12,Serviceid_125,...,User_id & Date_4958104 2018-02-12,User_id & Date_4958104 2018-02-13,User_id & Date_4958104 2018-02-14,User_id & Date_4958104 2018-03-02,User_id & Date_4958104 2018-03-03,User_id & Date_4958104 2018-03-04,User_id & Date_4958104 2018-03-08,User_id & Date_4958104 2018-03-11,User_id & Date_4958104 2018-06-14,User_id & Date_4958104 2018-06-30
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
basket_sets = pd.pivot_table(basket, index='User_id & Date', aggfunc='sum')
basket_sets

KeyError: 'User_id & Date'

## Analysis Question 3:
Any ideas that you want to analyze more with those data?
1. Who, User_id, buy most in term of TotalService?
2. Which Serviceid that each Age_group buy most?
3. Which of the best combo of Serviceid we can offer for each Age_group?
4. Which Age_Group that perform lowest performance in buying service?
5. Which Age_Group that perform the highest performance in buying service?


# Analysis Test 2

## Question 1:
1. What should we do to increase the average number of scenarios (services) that users engage in?

        1a. Make an analysis plan to answer this question

In [51]:
# Step 1: Import the required libraries and read in the data set.
# Step 2: Data Exploration using df.head(), df.tail(), df.info().
# Step 3: Data Cleaning 
## Find the number of null values for all columns - df.isnull().sum()
## Find the number of duplicate data - df.duplicated().sum()
## Drop the duplicated rows - df.drop_duplicates(inplace = True)


In [50]:
# Find the basket_sets that each user engage in 
# Find the itemsets of each users which include 2 or more serviceid in each set.
# Find the frequent itemsets of each scenario then choose the first highest ones.
# From this table we can see that the itemsets that most used by customers 
# in order that we can make a combo of this sets to increase the  cross-selling of scenarios that users engage in.
# Recomment users of these itemsets via direct selling. 
# Make some promotion when buying the combo for more engagement.
# In addition, we can use the function group by to see which itemsets is most used by each age_group? 
# in order to offer the promotion for the itemsets(services) for Age_group segment.

    1b. What kind of customer data should we collect to better answer this questions?
    -> 'User_id' , 'serviceid', 'serviceid_number', 'date', 'user_age', 'user_age_group'.

## Question 2:
2. What should we do to increase the average frequency of transaction per users each month?
        2a. Make an analysis plan to answer this question

#Preparing and cleaning the data as above.
#Calculating repeat purchase rate = no. of users who bought more than once (in 30 days) / no. of total users (in 30 days)
#Calculating purchase frequency = no. of transations placed (in 30 days) / # of unique users (in 30 days)
#Calculating time between purchases = 30 days/ purchase frequency
#Use these data to find out these questions:

- How often does it make sense for users to be purchasing from this service? 
- What might be blocking users from purchasing more frequently? 
- Do any of marketing efforts or sales effort impact these numbers? 

#Action plans:
- Do email marketing: Ask users when they would like to repurchase and send them a reminder when it’s time. 
- Recommend new products based on purchase history. 
- Advertising: Retarget with an ad featuring a service that complements a customer’s most recent purchase. 
- Determine the right timeframe for customers to come back and purchase again and retarget customers who last purchased around that time. 
- Personalize the ads with their recently purchased items.
- Customer satisfaction: Identify points of friction, whether in the checkout, shipping, product use, or customer service process and create content and campaigns to help. 
- Build loyalty programs: Reward customers with points for every purchase maded, then they can use that points to make a discount for next purchase. 

        2b. What kind of customer data should we collect to better answer this questions?
         -> 'User_id', 'number of total user each month', 'users purchase more than once each month', 'servicedate', 'serviceid', 'number of serviceid per user each month'